**init environment**

In [1]:
!git clone https://github.com/thor9king/Realtime_voice_clone_mandarin.git
%cd /content/Realtime_voice_clone_mandarin
!pip install -r requirements.txt
!pip install pypinyin


Cloning into 'Realtime_voice_clone_mandarin'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 123 (delta 17), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (123/123), 15.27 MiB | 9.29 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/Realtime_voice_clone_mandarin
Ignoring numpy: markers 'platform_system == "Windows"' don't match your environment
     |████████████████████████████████| 86 kB 2.8 MB/s 
     |████████████████████████████████| 676 kB 27.4 MB/s 
     |████████████████████████████████| 10.3 MB 32.0 MB/s 
     |████████████████████████████████| 14.5 MB 65 kB/s 
     |████████████████████████████████| 235 kB 40.8 MB/s 
     |████████████████████████████████| 8.3 MB 40.1 MB/s 
     |████████████████████████████████| 66 kB 4.5 MB/s 
     |████████████████████████████████| 1.3 MB 49.7 MB/s 
     |████████████████████████████████| 1.1 MB 36.0 MB/s 
     

download the pretrained model

In [2]:
!gdown --id 1tpQMQ36qGg_SNWKbM0WNYEcKrUt84pAE
!gdown --id 174SieJdc4wOQSY3GtedH0aGfSaUTb8A8

Downloading...
From: https://drive.google.com/uc?id=1tpQMQ36qGg_SNWKbM0WNYEcKrUt84pAE
To: /content/Realtime_voice_clone_mandarin/mandarin_300k.pt
100% 371M/371M [00:02<00:00, 154MB/s]
Downloading...
From: https://drive.google.com/uc?id=174SieJdc4wOQSY3GtedH0aGfSaUTb8A8
To: /content/Realtime_voice_clone_mandarin/pretrained.pt
100% 53.8M/53.8M [00:00<00:00, 58.8MB/s]


In [3]:
from encoder.params_model import model_embedding_size as speaker_embedding_size
from utils.argutils import print_args

from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder

from pathlib import Path
import numpy as np
import soundfile as sf
import librosa
import argparse
import torch
import sys
import os

from audioread.exceptions import NoBackendError
from pypinyin import Style
from pypinyin.contrib.neutral_tone import NeutralToneWith5Mixin
from pypinyin.converter import DefaultConverter
from pypinyin.core import Pinyin

import matplotlib
import matplotlib.pylab as plt
import IPython.display as ipd
from IPython.display import Audio
class PinyinConverter(NeutralToneWith5Mixin, DefaultConverter):
    pass

pinyin = Pinyin(PinyinConverter()).pinyin

def py(input,style):
    resp = pinyin(input, style=style)
    res = [v[0] for v in resp if v[0].strip()]
    res = " ".join(res)
    return res

In [4]:
syn_model_dir="/content/Realtime_voice_clone_mandarin/mandarin_300k.pt"
enc_model_dir="/content/Realtime_voice_clone_mandarin/encoder/saved_models/pretrained.pt"
vocoder_model_dir="/content/Realtime_voice_clone_mandarin/pretrained.pt"

   
if torch.cuda.is_available():
    device_id = torch.cuda.current_device()
    gpu_properties = torch.cuda.get_device_properties(device_id)
    ## Print some environment information (for debugging purposes)
    print("Found %d GPUs available. Using GPU %d (%s) of compute capability %d.%d with "
        "%.1fGb total memory.\n" % 
        (torch.cuda.device_count(),
        device_id,
        gpu_properties.name,
        gpu_properties.major,
        gpu_properties.minor,
        gpu_properties.total_memory / 1e9))
else:
    print("Using CPU for inference.\n")
    
    ## Load the models one by one.
print("Preparing the encoder, the synthesizer and the vocoder...")
encoder.load_model(Path(enc_model_dir))
synthesizer = Synthesizer(Path(syn_model_dir))

vocoder.load_model(Path(vocoder_model_dir))

Found 1 GPUs available. Using GPU 0 (Tesla K80) of compute capability 3.7 with 12.0Gb total memory.

Preparing the encoder, the synthesizer and the vocoder...
Loaded encoder "pretrained.pt" trained to step 1564501
Synthesizer using device: cuda
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at /content/Realtime_voice_clone_mandarin/pretrained.pt


In [5]:
ref_sound_path="/content/Realtime_voice_clone_mandarin/samples/a_tf6910_258_yangguan.wav" #you can upload your own voice
from IPython.display import Audio
Audio(ref_sound_path)

In [6]:
input_text="山上有座庙，庙里有个和尚"

embed = encoder.embed_utterance(encoder.preprocess_wav(ref_sound_path))
res= py(input_text,Style.TONE3)
texts = [res]
embeds = [embed]

specs = synthesizer.synthesize_spectrograms(texts, embeds)
spec = specs[0]
print("Created the mel spectrogram")
print("Synthesizing the waveform:")
    # If seed is specified, reset torch seed and reload vocoder

    
generated_wav = vocoder.infer_waveform(spec)
generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
generated_wav = encoder.preprocess_wav(generated_wav)



Trainable Parameters: 30.875M
self.n_mels:  80
self.lstm_dims:  1024
self.encoder_dims:  256
self.decoder_dims: 128
Loaded synthesizer "mandarin_300k.pt" trained to step 300000
+----------+---+
| Tacotron | r |
+----------+---+
|   300k   | 2 |
+----------+---+
 

| Generating 1/1


Done.

Created the mel spectrogram
Synthesizing the waveform:
{| ████████████████ 57000/57600 | Batch Size: 6 | Gen Rate: 4.0kHz | }

In [7]:
ipd.Audio(generated_wav.astype(np.float32), rate=22050)